# GAP
The Generalized Assignment Problem (GAP) is a generalization of the Assignment Problem (AP) in which there are a set of agents and a set of tasks. Each agent is required to perform exactly one task. Each agent has a cost associated with performing a task. The objective is to minimize the total cost of assigning agents to tasks.

## Problem Statement
The GAP can be formally defined as follows:
```
Given a set of n agents and a set of m tasks, the cost of assigning agent i to task j is cij. The goal is to find an assignment of agents to tasks that minimizes the total cost.
```
Notations:
- n denotes the number of agents.
- m denotes the number of tasks.
- cij denotes the cost associated with assigning the ith agent to the jth task.
- rij denotes the resource required for assigning the ith agent to the jth task.
- ci denotes the capacity of the ith agent.

Decision Variables:
- xij = 1 if the ith agent is assigned to the jth task, 0 otherwise.

Formulation:
```python
Minimize: sum(sum(cij * xij for j in range(m)) for i in range(n))

Subject to:
sum(xij for i in range(n)) == 1, for all j in range(m)
sum(rij * xij for j in range(m)) <= ci, for all i in range(n)
xij in {0, 1}, for all i in range(n) and for all j in range(m)
```




In [ ]:
# read the data under GAP folder and generate the data for the model
import os
import numpy as np
import gurobipy as gp
from gurobipy import GRB

: 

In [39]:
# Helper function to extract 100 elements from multiple lines (12 per line)
def extract_full_row(start_idx, lines_needed, lines):
    elements = []
    for i in range(lines_needed):
        line_elements = list(map(int, lines[start_idx + i].split()))
        elements.extend(line_elements)
    return elements

In [89]:
def read_gap_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.read().splitlines()
        # Read the first line which contains m and n
        m, n = map(int, lines[0].split())

        lines_reqd = n // 12 + 1
        # Initialize lists to hold the resources and costs
        costs = []
        resources = []
        capacity_constraints = []

        # Extract costs (first m rows of 100 elements each)
        cost_start_idx = 1
        for i in range(m):
            costs.append(extract_full_row(cost_start_idx + i * lines_reqd, lines_reqd, lines))

        # Extract resources (next m rows of 100 elements each)
        resource_start_idx = cost_start_idx + m * (lines_reqd)
        for i in range(m):
            resources.append(extract_full_row(resource_start_idx + i * lines_reqd, lines_reqd, lines))

        # Extract the final capacity constraint values
        capacity_start_idx = resource_start_idx + m * (lines_reqd)
        capacity_constraints = list(map(int, ' '.join(lines[capacity_start_idx:]).split()))


        # Convert lists to numpy arrays (optional)
        costs = np.array(costs)
        resources = np.array(resources)
        capacity_constraints = np.array(capacity_constraints)

        # Print the extracted details
        # print("\nCosts:")
        # print(costs)

        # print("\nResources:")
        # print(resources)

        # print("\nCapacity Constraints:")
        # print(capacity_constraints)
        
        return m, n, costs, resources, capacity_constraints



In [96]:
def solve_gap(resource_matrix, cost_matrix, capacity, m, n, output_file="model.lp"):
    # Create a new model
    model = gp.Model("GAP")
    
    # Suppress the output (initial Gurobi information)
    model.setParam('OutputFlag', 0)

    # Create variables (continuous instead of binary)
    x = model.addVars(m, n, vtype=GRB.BINARY, name="x")

    # Set objective (minimize cost)
    model.setObjective(sum(cost_matrix[i][j] * x[i, j] for i in range(m) for j in range(n)), GRB.MINIMIZE)

    # Each task is assigned to exactly one resource (relaxed to continuous)
    model.addConstrs((x.sum('*', j) == 1 for j in range(n)), name="task")

    # Capacity constraints (keeping them as constraints)
    model.addConstrs((sum(resource_matrix[i][j] * x[i, j] for j in range(n)) <= capacity[i] for i in range(m)), name="capacity")

    # Optimize model
    model.optimize()

    # Print the objective value
    print(f"Objective value: {model.objVal}")


    # After solving, convert variables from binary to continuous and ensure bounds
    for var in model.getVars():
        var.vtype = GRB.CONTINUOUS  # Convert to continuous
        # var.lb = 0  # Ensure lower bound is >= 0
        # var.ub = 1  # Ensure upper bound is <= 1

    # Write the modified model with continuous variables to an .lp file
    model.update()  # Ensure the model reflects changes to the variable types
    model.write(output_file)

    return model


In [79]:
# Output LP files directory
output_dir = 'lp_files'
os.makedirs(output_dir, exist_ok=True)

In [ ]:
for root, dirs, files in os.walk("GAP"):
        for file in files:
            print(f"File: {file}")
            # if file != "gap1.txt":
            m, n, cost_matrix, resource_matrix, capacity = read_gap_file(os.path.join(root, file))
            print(f"m = {m}, n = {n}")
            model = solve_gap(resource_matrix, cost_matrix, capacity, m, n, os.path.join(output_dir, f"{file}.lp"))
            print()
            break

            

In [106]:
for root, dirs, files in os.walk("GAP"):
        for file in files:
            print(f"File: {file}")
            print(f"m = {m}, n = {n}")
            model = gp.read(os.path.join(output_dir, f"{file}.lp"))
            model.setParam('OutputFlag', 0)
            optimal_value, optimal_obj = branch_and_bound(model)
            print(f"Optimal value: {optimal_value}")
            print(f"Optimal objective: {optimal_obj}")
            print()
            break

File: a05100
m = 5, n = 100
Read LP format model from file lp_files\a05100.lp
Reading time = 0.00 seconds
: 105 rows, 500 columns, 1000 nonzeros
Optimal solution found with objective: 1698.0
Optimal value: <gurobi.Model Continuous instance _copy_copy_copy_copy_copy: 105 constrs, 500 vars, Parameter changes: OutputFlag=0>
Optimal objective: 1698.0



In [103]:
import gurobipy as gp
from gurobipy import GRB

def branch_and_bound(model, tolerance=1e-6):
    """
    A custom branch-and-bound method for solving a Mixed-Integer Linear Program (MILP).
    The method performs branching on variables and bounds the solutions by exploring the search space.

    Parameters:
        model: The Gurobi model object (with the continuous relaxation)
        tolerance: The tolerance level for deciding whether a variable is "close" to 0 or 1 (default: 1e-6)

    Returns:
        Optimal solution value and the model with updated variable assignments.
    """
    
    # Initialize the best solution and best objective
    best_solution = None
    best_objective = float('inf')
    
    def explore_node(model):
        """
        Recursively explores a branch of the search tree.
        
        Parameters:
            model: The current Gurobi model to explore (with potential variable bounds modified)

        Returns:
            None. Updates the best_solution and best_objective as it explores nodes.
        """
        nonlocal best_solution, best_objective
        
        # Solve the relaxed model
        model.optimize()
        
        # Check if the model has an optimal solution
        if model.status != GRB.OPTIMAL:
            return  # Stop exploring this branch if not feasible
        
        # If the objective is worse than the best known, stop exploring this branch
        obj_val = model.objVal
        if obj_val >= best_objective:
            return
        
        # Check if the solution is integral
        fractional_var = None
        for var in model.getVars():
            if tolerance < var.x < 1 - tolerance:
                fractional_var = var
                break
        
        if fractional_var is None:
            # If all variables are integral, check if this is the best solution
            best_solution = model.copy()
            best_objective = obj_val
            return
        
        # Otherwise, branch on the fractional variable
        # Create two child nodes: one with var <= 0 and the other with var >= 1
        var_name = fractional_var.varName
        lower_bound = fractional_var.lb
        upper_bound = fractional_var.ub
        
        # Left branch: var <= floor(var.x)
        model_left = model.copy()
        model_left.getVarByName(var_name).ub = 0
        explore_node(model_left)
        
        # Right branch: var >= ceil(var.x)
        model_right = model.copy()
        model_right.getVarByName(var_name).lb = 1
        explore_node(model_right)
    
    # Start branching with the root node (continuous relaxation)
    explore_node(model)
    
    if best_solution is not None:
        print("Optimal solution found with objective:", best_objective)
        return best_solution, best_objective
    else:
        print("No feasible solution found.")
        return None, None


# Sample usage:
# Assuming you have a Gurobi model built already, e.g., from a .lp file
# model = gp.read("path_to_your_lp_file.lp")
# optimal_model, optimal_obj = branch_and_bound(model)


In [ ]:
# read .lp file and solve it
# model = gp.read("model.lp")
# model.optimize()
# print(f"Objective value: {model.objVal}")
# model.write("model.lp")
